# [Thyroid Disease dataset](http://odds.cs.stonybrook.edu/thyroid-disease-dataset)


Dataset information
The original thyroid disease (ann-thyroid) dataset from UCI machine learning repository is a classification dataset, which is suited for training ANNs. It has 3772 training instances and 3428 testing instances. It has 15 categorical and 6 real attributes. The problem is to determine whether a patient referred to the clinic is hypothyroid. Therefore three classes are built: normal (not hypothyroid), hyperfunction and subnormal functioning. For outlier detection, 3772 training instances are used, with only 6 real attributes. The hyperfunction class is treated as outlier class and other two classes are inliers, because hyperfunction is a clear minority class.

Source (citation)
F. Keller, E. Muller, K. Bohm.“HiCS: High-contrast subspaces for density-based outlier ranking.” ICDE, 2012.

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.Downloads

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

Downloads
File: thyroid.mat

Description: X = Multi-dimensional point data, y = labels (1 = outliers, 0 = inliers)

# Import libraries

In [ ]:
pip install pyod

In [ ]:
pip install mlconfig

In [ ]:
pip install segmentation_models_3D

In [ ]:
!pip install git+https://github.com/jonbarron/robust_loss_pytorch


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
import torch
import mlconfig
import random as rn
import keras
import segmentation_models_3D as sm
import robust_loss_pytorch 
import shutil


from pathlib import Path
from time import strftime
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)
from sklearn.metrics import confusion_matrix,accuracy_score,cohen_kappa_score,roc_auc_score,f1_score,auc

Achtung: Alle Dateien wurden in einem Ordner auf Drive gespeichert. Um den Code zu replizieren empfehle ich, ebenfalls einen Ordner auf Drive zu erstellen und folgendermaßen das working directory anzupassen:

```
from google.colab import drive
drive.mount('/content/drive')
```

```
cd /content/drive/MyDrive/<specify here your path>
```



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys

assert sys.version_info >= (3, 7) #python 3.7 or higher recommended

In [ ]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1") #sklearn 1.01 or higher remmonded

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


assert version.parse(tf.__version__) >= version.parse("2.8.0") #tensorflow 2.8.0 or higher remmonded 

Standardeinstellungen für Plots anpassen

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:
cd /content/drive/MyDrive/Seminar2/ADAPL #specify here your path

Erstellen eines Bildordners zum Sammeln aller Bildausgaben

Erstellen Sie eine Funktion zum automatischen Speichern eines Plots in einem bestimmten Pfad.

In [ ]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "thyroid"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
ls

In [ ]:
tf.keras.backend.clear_session()

# Data Wrapping 

Lade .mat Datei und konventiere zu pandas Dataframe

In [ ]:
import scipy.io

data_dict = scipy.io.loadmat("thyroid.mat")
data_dict

In [ ]:
data = []
for key, values in data_dict.items():
  data.append(values)
data = data[3:]
df = data[0]
labels = data[1]
df = pd.DataFrame(data=df)
df["label"] = labels

df

In [ ]:
df.describe()

In [ ]:
df.info()

# Data Cleaning

In [ ]:
print(f'Data size is {df.shape}.')

In [ ]:
df.dtypes

In [ ]:
#check for NA-Values 
df.isnull().sum()

Keine NA vorhanden

# EDA and Visualization

In [ ]:
#print boxplot for each variable
fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in df.items():
    sns.boxplot(y=k, data=df, ax=axs[index])
    index += 1
save_fig("boxplots")
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

In [ ]:
for k, v in df.items():
        q1 = v.quantile(0.25)
        q3 = v.quantile(0.75)
        irq = q3 - q1
        v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
        perc = np.shape(v_col)[0] * 100.0 / np.shape(df)[0]
        print("Column %s outliers = %.2f%%" % (k, perc))

In [ ]:
#print distributions
fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in df.items():
    sns.distplot(v, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
save_fig("distrubutions")

In [ ]:
df['label'].value_counts().plot(kind='bar', figsize=(8, 4));
plt.title('Distrubution of outliers');
plt.xlabel('Diagnosis');
plt.ylabel('Frequency');
plt.xticks([0.0, 1.0], ['Normal', 'Anomaly'], rotation=45);
save_fig("distrubution outliers")

In [ ]:
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sn

sns.set(style='whitegrid', context='notebook')

def tsne_scatter(features, labels, dimensions=2, save_as='graph.png', RANDOM_SEED = 42):
    if dimensions not in (2, 3):
        raise ValueError('tsne_scatter can only plot in 2d or 3d')

    # t-SNE dimensionality reduction
    features_embedded = TSNE(n_components=dimensions, random_state=RANDOM_SEED).fit_transform(features)
    
    # initialising the plot
    fig, ax = plt.subplots(figsize=(8,8))
    
    # counting dimensions
    if dimensions == 3: ax = fig.add_subplot(111, projection='3d')

    # plotting data
    ax.scatter(
        *zip(*features_embedded[np.where(labels==1)]),
        marker='o',
        color='r',
        s=2,
        alpha=0.7,
        label='Anomaly'
    )
    ax.scatter(
        *zip(*features_embedded[np.where(labels==0)]),
        marker='o',
        color='g',
        s=2,
        alpha=0.3,
        label='Normal'
    )

    # storing it to be displayed later
    sns.set(style='whitegrid', context='notebook')
    save_fig(save_as)
    plt.show;

In [ ]:
df['anomaly'] = df['label'] == 1.0
anomaly = df[df['anomaly'] == True]
normal = df[df['anomaly'] == False]

In [ ]:
sns.distplot(normal);
sns.distplot(anomaly);

plt.title('normal vs  anomaly Dist.');
plt.ylabel('Dist.');
save_fig("normal vs anomaly distrubution")

# Normalize The Data

In [ ]:
data = df.iloc[:, 0:6]
data

In [ ]:
# The last element contains the labels
labels_bool = df.iloc[:,-1]
labels  = df.iloc[:,-2]
# The other data points are the features
data = df.iloc[:, 0:6]

data = normalize(data) #normalize data
n_features = 6

In [ ]:
data

In [ ]:
tsne_scatter(data, labels, dimensions=2, save_as='tsne_initial_2d')
tsne_scatter(data, labels, dimensions=3, save_as='tsne_initial_3d')

Die Anomalien werden mit T-SNE zum Großteil in Cluster eingeteilt. 

# Split the data

Generiere Trainings- und Testdaten

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels_bool,
    test_size=0.2,
    
)

Generiere das Validation-Set

In [ ]:
train_data_no_validation, train_data_validation, train_data_no_validation_labels, train_data_validation_labels = train_test_split(
    train_data,
    train_labels,
    test_size = 0.2,
)
# only train_data_validation and train_data_validation_labels are needed for further analysis

In [ ]:
anomalous_train_data = train_data[train_labels] #training data with outlier-label == True
anomalous_test_data = test_data[test_labels] #test data with outlier-label == True

normal_train_data = train_data[~train_labels] #training data with outlier-label == False
normal_test_data = test_data[~test_labels] #test data with outlier-label == False

In [ ]:
for k in [train_data, train_labels, train_data_validation, train_data_validation_labels]: 
 print(k.shape)

In [ ]:
for k in [train_data, test_data, train_labels, test_labels]: 
 print(k.shape)

In [ ]:
for k in [anomalous_train_data, anomalous_test_data, normal_train_data, normal_test_data]: 
 print(k.shape)

In [ ]:
print("Anomaly Share for training data equals ={:10.4f}".format(len(anomalous_train_data)/len(train_data)))
print("Anomaly Share for test data equals ={:10.4f}".format(len(anomalous_test_data)/len(test_data)))

# Train the model with train & validation data

## Randomized autoencoders

Die Implementierung für die RandAE-Klasse stammt aus folgendem [Repository](https://github.com/danieltsoukup/autoencoders):

Aktivierungsfunktionen

*   Hidden Layer: Relu
*   Output Layer: Sigmoid

Es wird ein Drop_ratio von 0.5 festgelegt. Ein Drop_ratio von 0.0 entspricht einer *fully connected architecture*



In [ ]:
class RandAE(tf.keras.Sequential):
    def __init__(self, input_dim, hidden_dims, drop_ratio=0.5, **kwargs):
        super(RandAE, self).__init__(**kwargs)
        
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.drop_ratio = drop_ratio
        
        self.layer_masks = dict()
        
        self.build_model()
                
    def build_model(self) -> None:
        """
        Adds the layers and records masks.
        """
        
        self.add(layers.Input(self.input_dim, name="input"))
        
        for i, dim in enumerate(self.hidden_dims):
            layer_name = f"hidden_{i}"
            layer = layers.Dense(dim, 
                                 activation="relu" if i > 0 else "sigmoid", 
                                 name=layer_name)
            self.add(layer)
            
            # add layer mask
            self.layer_masks[layer_name] = self.get_mask(layer)
        
        layer_name = "output"
        output_layer = layers.Dense(self.input_dim, activation="sigmoid", name=layer_name)
        self.add(output_layer)
        self.layer_masks[layer_name] = self.get_mask(output_layer)
            
    def get_mask(self, layer) -> np.ndarray:
        """
        Build mask for a layer.
        """
        
        shape = layer.input_shape[1], layer.output_shape[1]
        
        return np.random.choice([0., 1.], size=shape, p=[self.drop_ratio, 1-self.drop_ratio])
        
    def load_masks(self, mask_pickle_path) -> None:
        """
        Load the masks from a pickled dictionary.
        """
        
        with open(mask_pickle_path, 'rb') as handle:
            self.layer_masks = pickle.load(handle)    
            
    def get_encoder(self) -> keras.Sequential:
        """
        Get the encoder from the full model.
        """
        
        n_layers = (len(self.hidden_dims)+1)//2
        encoder_layers = [layers.Input(self.input_dim)] + self.layers[:n_layers]

        return keras.Sequential(encoder_layers)
        
    
    def mask_weights(self) -> None:
        """
        Apply the masks to each layer in the encoder and decoder.
        """
        
        for layer in self.layers:
            layer_name = layer.name
            if layer_name in self.layer_masks:
                masked_w = layer.weights[0].numpy()*self.layer_masks[layer_name]
                b = layer.weights[1].numpy()
                layer.set_weights((masked_w, b))        

    def call(self, data, training=True) -> tf.Tensor:
        
        # mask the weights before original forward pass
        self.mask_weights()
        
        return super().call(data)

In [ ]:
model_mse = RandAE(6,[24,12,6,3,6,12,24])
model_mae = RandAE(6,[24,12,6,3,6,12,24])
model_ce = RandAE(6,[24,12,6,3,6,12,24])
model_focal = RandAE(6,[24,12,6,3,6,12,24])
model_huber = RandAE(6,[24,12,6,3,6,12,24])

## Training

Trainiere das Modell mit dem MSE

In [ ]:
#load checkpoints
# checkpoint_mse = tf.keras.callbacks.ModelCheckpoint("checkpoint_mse",
#                                                    save_weights_only=True) #generate checkpoints


#define early stopping
early_stopping_mse = tf.keras.callbacks.EarlyStopping(patience=15,
                                                     restore_best_weights=True) 



                                                                                                                                                       
model_mse.compile(loss="mean_squared_error", optimizer="adam", run_eagerly=True)


history_mse = model_mse.fit(train_data,
                    train_data,
                    batch_size=16,
                    epochs = 1000,
                    validation_data=(train_data_validation, train_data_validation),
                    callbacks=[
                        # checkpoint_mse,
                        early_stopping_mse,
                        ],
                    verbose = 0,
                    validation_batch_size = 16,
                    shuffle=True)
model_mse.summary()

In [ ]:
# checkpoint_mae = tf.keras.callbacks.ModelCheckpoint("checkpoint_mae",
#                                                    save_weights_only=True) #generate checkpoints
early_stopping_mae = tf.keras.callbacks.EarlyStopping(patience=15,
                                                     restore_best_weights=True) #define early stopping                                                  
model_mae.compile(loss="mean_absolute_error", optimizer="adam", run_eagerly=True)
history_mae = model_mae.fit(train_data,
                    train_data,
                    batch_size=16,
                    epochs = 1000,
                    validation_data=(train_data_validation, train_data_validation),
                    callbacks=[
                        # checkpoint_mae,
                        early_stopping_mae],
                    verbose = 0,
                    validation_batch_size = 16,
                    shuffle=True)
model_mae.summary()

In [ ]:

# checkpoint_ce = tf.keras.callbacks.ModelCheckpoint("checkpoint_ce",
#                                                    save_weights_only=True) #generate checkpoints
early_stopping_ce = tf.keras.callbacks.EarlyStopping(patience=15,
                                                     restore_best_weights=True) #define early stopping

model_ce.compile(loss="binary_crossentropy", optimizer="adam",  run_eagerly=True)

history_ce = model_ce.fit(train_data,
                    train_data,
                    batch_size=16,
                    epochs = 1000,
                    validation_data=(train_data_validation, train_data_validation),
                    callbacks=[
                        # checkpoint_ce,
                        early_stopping_ce],
                    verbose = 0,
                    validation_batch_size = 16,
                    shuffle=True)
model_ce.summary()

Der Focal Loss wird mit γ = 2 initalisiert.

[Quelle](https://github.com/qubvel/segmentation_models/blob/master/segmentation_models/losses.py)

In [ ]:
focal_loss = sm.losses.BinaryFocalLoss()
# checkpoint_focal = tf.keras.callbacks.ModelCheckpoint("checkpoint_focal",
#                                                    save_weights_only=True) #generate checkpoints
early_stopping_focal = tf.keras.callbacks.EarlyStopping(patience=15,
                                                     restore_best_weights=True) #define early stopping

model_focal.compile(loss=focal_loss, optimizer="adam",  run_eagerly=True)


history_focal = model_focal.fit(train_data,
                                train_data,
                                batch_size=16,
                                epochs = 1000,
                    validation_data=(train_data_validation, train_data_validation),
                    callbacks=[
                        # checkpoint_focal,
                        early_stopping_focal],
                    verbose = 0,
                    validation_batch_size = 16,
                    shuffle=True)
model_focal.summary()

Huber wird mit δ = 0,5 initalisiert

In [ ]:
huber = tf.keras.losses.Huber(delta= 0.5, reduction="auto", name="huber_loss")
# checkpoint_huber = tf.keras.callbacks.ModelCheckpoint("checkpoint_huber",
#                                                    save_weights_only=True) #generate checkpoints
early_stopping_huber = tf.keras.callbacks.EarlyStopping(patience=15,
                                                     restore_best_weights=True) #define early stopping

model_huber.compile(loss=huber, optimizer="adam",  run_eagerly=True) 

history_huber = model_huber.fit(train_data,
                    train_data,
                    batch_size=16,
                    epochs = 1000,
                    validation_data=(train_data_validation, train_data_validation),
                    callbacks=[
                        #checkpoint_huber,
                        early_stopping_huber],
                    verbose = 0,
                    validation_batch_size = 16,
                    shuffle=True)
model_huber.summary()

Plotte den Trainingsverlauf für jedes Modell

In [ ]:
def plot_training_performance():
    '''
    Plot the training performance of each model 
    '''
    figure, axis = plt.subplots(ncols=1,nrows=5, figsize=(10, 20))
    # first row: complicated architecture
    axis[0].plot(history_mse.history["loss"], label="Training Loss")
    axis[0].plot(history_mse.history["val_loss"], label="Validation Loss")
    axis[0].set_title('Performance Training with model_mse')
    axis[0].set(xlabel='Epochs', ylabel='Loss')
    axis[0].legend()
    
    axis[1].plot(history_mae.history["loss"], label="Training Loss")
    axis[1].plot(history_mae.history["val_loss"], label="Validation Loss")
    axis[1].set_title('Performance Training with model_mae')
    axis[1].set(xlabel='Epochs', ylabel='Loss')
    axis[1].legend()
    
    axis[2].plot(history_huber.history["loss"], label="Training Loss")
    axis[2].plot(history_huber.history["val_loss"], label="Validation Loss")
    axis[2].set_title('Performance Training with model_huber')
    axis[2].set(xlabel='Epochs', ylabel='Loss')
    axis[2].legend()
    
    #second row: simple architectures 
    axis[3].plot(history_ce.history["loss"], label="Training Loss")
    axis[3].plot(history_ce.history["val_loss"], label="Validation Loss")
    axis[3].set_title('Performance Training by model_ce')
    axis[3].set(xlabel='Epochs', ylabel='Loss')
    axis[3].legend()
    
    axis[4].plot(history_focal.history["loss"], label="Training Loss")
    axis[4].plot(history_focal.history["val_loss"], label="Validation Loss")
    axis[4].set_title('Performance Training by model_focal')
    axis[4].set(xlabel='Epochs', ylabel='Loss')
    axis[4].legend()

    save_fig("Performances for training")
    plt.show()

plot_training_performance()

Diese Funktion extrahiert die letzte ausgeführte Epoche für ein Model

In [ ]:
def get_last_epoch(history, max = "max") -> int:
  epochs = np.array([])
  stopped_epoch = np.array([])
  for key, value in enumerate(history.history["loss"]):
    epochs = np.append(epochs,key)
  stopped_epoch = np.append(stopped_epoch,int(np.max(epochs)))
  if max == "max":
    return int(np.max(epochs))
  else:
    return epochs


In [ ]:
for history in [
history_mse,
    history_mae,
    history_ce,
    history_focal,
    history_huber ]:
    print(get_last_epoch(history))

Plotte den TNSE-Graph für den latenten Raum in einem Modell

In [ ]:
def plot_tnse_performance_bottleneck_layer():
    '''
    Plot the training performance of each model 
    '''
    figure, axis = plt.subplots(ncols=1,nrows=5, figsize=(10, 20))
    # first row: complicated architecture
    encoder = model_mse.get_encoder()
    vis_data_latent = encoder.predict(train_data)
    tsne = TSNE(n_components=2)
    tsne_data = tsne.fit_transform(vis_data_latent)
    axis[0].scatter(tsne_data[train_labels == 0, 0], 
            tsne_data[train_labels == 0, 1], c="grey", alpha=0.1, label="inlier")
    axis[0].scatter(tsne_data[train_labels == 1, 0], 
            tsne_data[train_labels == 1, 1], c="crimson", alpha=1, label="outlier")
    axis[0].set_title('t-SNE for outliers/inliers on the latent manifold for model_mse')
    axis[0].legend()
    
    encoder = model_mae.get_encoder()
    vis_data_latent = encoder.predict(train_data)
    tsne = TSNE(n_components=2)
    tsne_data = tsne.fit_transform(vis_data_latent)
    axis[1].scatter(tsne_data[train_labels == 0, 0], 
            tsne_data[train_labels == 0, 1], c="grey", alpha=0.1, label="inlier")
    axis[1].scatter(tsne_data[train_labels == 1, 0], 
            tsne_data[train_labels == 1, 1], c="crimson", alpha=1, label="outlier")
    axis[1].set_title('t-SNE for outliers/inliers on the latent manifold for model_mae')
    axis[1].legend()
    
    encoder = model_huber.get_encoder()
    vis_data_latent = encoder.predict(train_data)
    tsne = TSNE(n_components=2)
    tsne_data = tsne.fit_transform(vis_data_latent)
    axis[2].scatter(tsne_data[train_labels == 0, 0], 
            tsne_data[train_labels == 0, 1], c="grey", alpha=0.1, label="inlier")
    axis[2].scatter(tsne_data[train_labels == 1, 0], 
            tsne_data[train_labels == 1, 1], c="crimson", alpha=1, label="outlier")
    axis[2].set_title('t-SNE for outliers/inliers on the latent manifold for model_huber')
    axis[2].legend()
    
    encoder = model_ce.get_encoder()
    vis_data_latent = encoder.predict(train_data)
    tsne = TSNE(n_components=2)
    tsne_data = tsne.fit_transform(vis_data_latent)
    axis[3].scatter(tsne_data[train_labels == 0, 0], 
            tsne_data[train_labels == 0, 1], c="grey", alpha=0.1, label="inlier")
    axis[3].scatter(tsne_data[train_labels == 1, 0], 
            tsne_data[train_labels == 1, 1], c="crimson", alpha=1, label="outlier")
    axis[3].set_title('t-SNE for outliers/inliers on the latent manifold for model_ce')
    axis[3].legend()
    
    encoder = model_focal.get_encoder()
    vis_data_latent = encoder.predict(train_data)
    tsne = TSNE(n_components=2)
    tsne_data = tsne.fit_transform(vis_data_latent)
    axis[4].scatter(tsne_data[train_labels == 0, 0], 
            tsne_data[train_labels == 0, 1], c="grey", alpha=0.1, label="inlier")
    axis[4].scatter(tsne_data[train_labels == 1, 0], 
            tsne_data[train_labels == 1, 1], c="crimson", alpha=1, label="outlier")
    axis[4].set_title('t-SNE for outliers/inliers on the latent manifold for model_focal')
    axis[4].legend()

    save_fig("TNSE-Distrubution in bottleneck-layer for all models")
    plt.show()

plot_tnse_performance_bottleneck_layer()

# Evaluation on test data

## Generiere den Rekonstruktionsfehler

Um den Rekonstruktionsfehler zu berechnen, wird als Metrik der MSE verwendet.

In [ ]:
def gen_error_df(model, metric:str = "mean_squared_error", data = test_data):
  if model in [model_mse,model_mae,model_ce,model_focal,model_huber]:
    print("Calculate reconstruction error for model: ", model)
    if metric == "mean_squared_error":
      mse = np.mean(np.power(data - model.predict(data), 2), axis=1)
      return(pd.DataFrame({'reconstruction_error': mse,
                              'anomaly': test_labels}))
    else:
      print("No further metric functions are implemented yet")
      return
  else:
    raise("Choosen model is not supported yet. Please choose an model in [model_mse,model_mae,model_ce,model_focal,model_huber]")




In [ ]:
mse_error_df = gen_error_df(model_mse)
mae_error_df = gen_error_df(model_mae)
ce_error_df = gen_error_df(model_ce)
focal_error_df = gen_error_df(model_focal)
huber_error_df = gen_error_df(model_huber)

## ROC und AUC

Visualisiere das Ergebnis für die Testdaten für alle Modelle

In [ ]:
def plot_test_performance_roc(error_df = [mse_error_df,
                                          mae_error_df,
                                          huber_error_df,
                                          ce_error_df,
                                          focal_error_df]):
    '''
    Plot the ROC curve performance of each model 
    '''
    figure, axis = plt.subplots(ncols=1,nrows=5, figsize=(10, 20))
    # first row: complicated architecture
    fpr, tpr, thresholds = roc_curve(error_df[0].anomaly, error_df[0].reconstruction_error)
    roc_auc = auc(fpr, tpr)
    axis[0].plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
    axis[0].plot([0,1],[0,1],'r--')
    axis[0].legend(loc='lower right')
    axis[0].set_title('Receiver Operating Characteristic for Model_MSE')
    axis[0].set_xlim([-0.001, 1])
    axis[0].set_ylim([0, 1.001])
    axis[0].set(xlabel='False Positive Rate', ylabel='True Positive Rate')
    axis[0].legend()
    
    fpr, tpr, thresholds = roc_curve(error_df[1].anomaly, error_df[1].reconstruction_error)
    roc_auc = auc(fpr, tpr)
    axis[1].plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
    axis[1].plot([0,1],[0,1],'r--')
    axis[1].legend(loc='lower right')
    axis[1].set_title('Receiver Operating Characteristic for Model_MAE')
    axis[1].set_xlim([-0.001, 1])
    axis[1].set_ylim([0, 1.001])
    axis[1].set(xlabel='False Positive Rate', ylabel='True Positive Rate')
    axis[1].legend()
    
    fpr, tpr, thresholds = roc_curve(error_df[2].anomaly, error_df[2].reconstruction_error)
    roc_auc = auc(fpr, tpr)
    axis[2].plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
    axis[2].plot([0,1],[0,1],'r--')
    axis[2].legend(loc='lower right')
    axis[2].set_title('Receiver Operating Characteristic for Model_Huber')
    axis[2].set_xlim([-0.001, 1])
    axis[2].set_ylim([0, 1.001])
    axis[2].set(xlabel='False Positive Rate', ylabel='True Positive Rate')
    axis[2].legend()
    
    #second row: simple architectures 
    fpr, tpr, thresholds = roc_curve(error_df[3].anomaly, error_df[3].reconstruction_error)
    roc_auc = auc(fpr, tpr)
    axis[3].plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
    axis[3].plot([0,1],[0,1],'r--')
    axis[3].legend(loc='lower right')
    axis[3].set_title('Receiver Operating Characteristic for Model_CE')
    axis[3].set_xlim([-0.001, 1])
    axis[3].set_ylim([0, 1.001])
    axis[3].set(xlabel='False Positive Rate', ylabel='True Positive Rate')
    axis[3].legend()
    
    fpr, tpr, thresholds = roc_curve(error_df[4].anomaly, error_df[4].reconstruction_error)
    roc_auc = auc(fpr, tpr)
    axis[4].plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
    axis[4].plot([0,1],[0,1],'r--')
    axis[4].legend(loc='lower right')
    axis[4].set_title('Receiver Operating Characteristic for Model_Focal')
    axis[4].set_xlim([-0.001, 1])
    axis[4].set_ylim([0, 1.001])
    axis[4].set(xlabel='False Positive Rate', ylabel='True Positive Rate')
    axis[4].legend()


    save_fig("Roc-Curves for test data")
    plt.show()

plot_test_performance_roc()

## Precision, Recall und F1-Score

Da diese Metriken davon abhängig sind, welche Threshold man für die Klassifikation wählt, wird die Threshold wie folgt gewählt, um den F1-Score zu maximieren:


*   Bestimme die Threshold, für die die Precision und der Recall gleich sind
*   Da es sich bei dem F1-Score um ein harmonisches Mittel handelt, wird so der F1-Score maximiert



In [ ]:
prec, recall, thresholds = precision_recall_curve(focal_error_df["anomaly"], focal_error_df["reconstruction_error"])

best_idx = np.argmin(np.abs(prec-recall)[:int(len(np.abs(prec-recall))*-0.1)]) #to resolve an bug, the last four obsvervations have to be excluded
best_prec, best_recall = prec[best_idx], recall[best_idx]
print(f"Best precision {np.round(best_prec, 2)}, "\
      f"recall: {np.round(best_recall, 2)} at {np.round(thresholds[best_idx], 2)} threshold.")

In [ ]:
from sklearn.metrics import precision_recall_curve, auc
auc_score = auc(recall, prec)
auc_score

Visualisiere das Ergebnis für alle Ergebnisse

In [ ]:
## Recall Vs Precision
def plot_test_performance_recall_vs_precision(error_df = [mse_error_df,
                                          mae_error_df,
                                          huber_error_df,
                                          ce_error_df,
                                          focal_error_df]):
    figure, axis = plt.subplots(ncols=2,nrows=5, figsize=(30, 30))
    # first row: complicated architecture
    precision, recall, thresholds = precision_recall_curve(error_df[0].anomaly, error_df[0].reconstruction_error)
    best_idx = np.argmin(np.abs(precision-recall)[:-4])
    best_prec, best_recall = precision[best_idx], recall[best_idx]
    precision_recall_auc = auc(recall, precision)
    axis[0,0].plot(recall, precision, label='AUC = %0.4f'% precision_recall_auc)
    axis[0,0].legend(loc='lower right')
    axis[0,0].set_title('Recall vs Precision for Model_MSE')
    axis[0,0].set(xlabel='Recall', ylabel='Precision')
    axis[0,1].set_title('Determine the threshold for Model_MSE')
    axis[0,1].plot(thresholds, precision[:-1], label="precision")
    axis[0,1].plot(thresholds, recall[:-1], label="recall")
    axis[0,1].axvline(thresholds[best_idx], 0, 1, c="grey", linestyle="--")
    axis[0,1].set_xlabel("thresholds")
    axis[0,1].legend()

    precision, recall, thresholds = precision_recall_curve(error_df[1].anomaly, error_df[1].reconstruction_error)
    best_idx = np.argmin(np.abs(precision-recall)[:-4])
    best_prec, best_recall = precision[best_idx], recall[best_idx]
    precision_recall_auc = auc(recall, precision)
    axis[1,0].plot(recall, precision, label='AUC = %0.4f'% precision_recall_auc)
    axis[1,0].legend(loc='lower right')
    axis[1,0].set_title('Recall vs Precision for Model_MAE')
    axis[1,0].set(xlabel='Recall', ylabel='Precision')
    axis[1,1].set_title('Determine the threshold for Model_MAE')
    axis[1,1].plot(thresholds, precision[:-1], label="precision")
    axis[1,1].plot(thresholds, recall[:-1], label="recall")
    axis[1,1].axvline(thresholds[best_idx], 0, 1, c="grey", linestyle="--")
    axis[1,1].set_xlabel("thresholds")
    axis[1,1].legend()

    precision, recall, thresholds = precision_recall_curve(error_df[2].anomaly, error_df[2].reconstruction_error)
    best_idx = np.argmin(np.abs(precision-recall)[:-4])
    best_prec, best_recall = precision[best_idx], recall[best_idx]
    precision_recall_auc = auc(recall, precision)
    axis[2,0].plot(recall, precision, label='AUC = %0.4f'% precision_recall_auc)
    axis[2,0].legend(loc='lower right')
    axis[2,0].set_title('Recall vs Precision for Model_Huber')
    axis[2,0].set(xlabel='Recall', ylabel='Precision')
    axis[2,1].set_title('Determine the threshold for Model_Huber')
    axis[2,1].plot(thresholds, precision[:-1], label="precision")
    axis[2,1].plot(thresholds, recall[:-1], label="recall")
    axis[2,1].axvline(thresholds[best_idx], 0, 1, c="grey", linestyle="--")
    axis[2,1].set_xlabel("thresholds")
    axis[2,1].legend()

    precision, recall, thresholds = precision_recall_curve(error_df[3].anomaly, error_df[3].reconstruction_error)
    best_idx = np.argmin(np.abs(precision-recall)[:-4])
    best_prec, best_recall = precision[best_idx], recall[best_idx]
    precision_recall_auc = auc(recall, precision)
    axis[3,0].plot(recall, precision, label='AUC = %0.4f'% precision_recall_auc)
    axis[3,0].legend(loc='lower right')
    axis[3,0].set_title('Recall vs Precision for Model_CE')
    axis[3,0].set(xlabel='Recall', ylabel='Precision')
    axis[3,1].set_title('Determine the threshold for Model_CE')
    axis[3,1].plot(thresholds, precision[:-1], label="precision")
    axis[3,1].plot(thresholds, recall[:-1], label="recall")
    axis[3,1].axvline(thresholds[best_idx], 0, 1, c="grey", linestyle="--")
    axis[3,1].set_xlabel("thresholds")
    axis[3,1].legend()

    precision, recall, thresholds = precision_recall_curve(error_df[4].anomaly, error_df[4].reconstruction_error)
    best_idx = np.argmin(np.abs(precision-recall)[:-4])
    best_prec, best_recall = precision[best_idx], recall[best_idx]
    precision_recall_auc = auc(recall, precision)
    axis[4,0].plot(recall, precision, label='AUC = %0.4f'% precision_recall_auc)
    axis[4,0].legend(loc='lower right')
    axis[4,0].set_title('Recall vs Precision for Model_FL')
    axis[4,0].set(xlabel='Recall', ylabel='Precision')
    axis[4,1].set_title('Determine the threshold for Model_Focal')
    axis[4,1].plot(thresholds, precision[:-1], label="precision")
    axis[4,1].plot(thresholds, recall[:-1], label="recall")
    axis[4,1].axvline(thresholds[best_idx], 0, 1, c="grey", linestyle="--")
    axis[4,1].set_xlabel("thresholds")
    axis[4,1].legend()

    save_fig("Recall vs Precision curves for test data")
    sns.despine()
    plt.show()

plot_test_performance_recall_vs_precision()

In [ ]:
def get_precision_threshold(error_df):
  '''
  Return an array, which includes the thresholds with the second highest precision score for each model
  '''
  prec, recall, threshold = precision_recall_curve(error_df["anomaly"], error_df["reconstruction_error"])
  best_idx = np.argmin(np.abs(prec-recall)[:-4])
  best_prec, best_recall = prec[best_idx], recall[best_idx]

  return threshold[best_idx]

for i in [mse_error_df, mae_error_df, huber_error_df, ce_error_df, focal_error_df]:   
  print(get_precision_threshold(i))

In [ ]:
def plot_thresholds_data(index:int,save_name:str,error_df = [mse_error_df,
                                          mae_error_df,
                                          huber_error_df,
                                          ce_error_df,
                                          focal_error_df]):
  if index > len(error_df) - 1 or index < 0:
    raise("Index out of range implemented. Please correct your statement")
  else:
    groups = error_df[index].groupby('anomaly')
    threshold = get_precision_threshold(error_df[index])
    fig, ax = plt.subplots()
    for name, group in groups:
      ax.plot(group.index, group.reconstruction_error, marker='o', ms=3.5, linestyle='',
              label= "Anomaly" if name == 1 else "Normal")
    ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
    ax.legend()
    plt.title("Reconstruction error visualisation to identify anomalies")
    plt.ylabel("Reconstruction error")
    plt.xlabel("Data point index")
    save_fig(save_name)
    plt.show();


In [ ]:
plot_thresholds_data(0, "Reconstruction error visualisation to identify anomalies with model_mse")
plot_thresholds_data(1, "Reconstruction error visualisation to identify anomalies with model_mae")
plot_thresholds_data(2, "Reconstruction error visualisation to identify anomalies with model_huber")
plot_thresholds_data(3, "Reconstruction error visualisation to identify anomalies with model_ce")
plot_thresholds_data(4, "Reconstruction error visualisation to identify anomalies with model_focal")

In [ ]:
## Predict class based on error threshold

# Weitermachen
def plot_cm_matrix(index:int,save_name:str,error_df = [mse_error_df,
                                          mae_error_df,
                                          huber_error_df,
                                          ce_error_df,
                                          focal_error_df],
                   LABELS = ["Non Fraud", "Fraud"]):
  if index > len(error_df) - 1 or index < 0:
    raise("Index out of range implemented. Please correct your statement")
  else:
    error_df = error_df[index]
    threshold = get_precision_threshold(error_df)
    y_pred = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
    conf_matrix = confusion_matrix(error_df.anomaly, y_pred,labels=[0,1])
    plt.figure(figsize=(12, 12))
    sns.heatmap(conf_matrix,xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d",cmap='Blues');
    plt.title("Confusion matrix")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    save_fig(save_name)
    plt.show()


In [ ]:
plot_cm_matrix(0, "CM-matrix visualisation to identify anomalies with model_mse")
plot_cm_matrix(1, "CM-matrix visualisation to identify anomalies with model_mae")
plot_cm_matrix(2, "CM-matrix visualisation to identify anomalies with model_huber")
plot_cm_matrix(3, "CM-matrix visualisation to identify anomalies with model_ce")
plot_cm_matrix(4, "CM-matrix visualisation to identify anomalies with model_focal")

In [ ]:
def calculate_f1_scores(index, error_df =  
                        [mse_error_df,
                         mae_error_df,
                         huber_error_df,
                         ce_error_df,
                         focal_error_df]
                        ):
  if index > len(error_df) - 1 or index < 0:
    raise("Index out of range implemented. Please correct your statement")
  error_df = error_df[index]
  threshold = get_precision_threshold(error_df)
  y_pred =  [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
  print('F1_Score with threshold:', threshold)
  return(f1_score(error_df.anomaly, y_pred))
calculate_f1_scores(0)

In [ ]:
error_df = [mse_error_df,
             mae_error_df,
             huber_error_df,
             ce_error_df,
             focal_error_df]
for i in error_df:
  threshold = get_precision_threshold(i) #specify here the index 
  y_pred =  [1 if e > threshold else 0 for e in i.reconstruction_error.values]

  cm1 = confusion_matrix(i.anomaly, y_pred,labels=[1,0])
  print('Confusion Matrix Val: \n', cm1)


  total1=sum(sum(cm1))
  #####from confusion matrix calculate accuracy

  accuracy1=(cm1[0,0]+cm1[1,1])/total1
  print ('Accuracy Val: ', accuracy1)


  sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  print('Sensitivity Val: ', sensitivity1 )


  specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  print('Specificity Val: ', specificity1)

  KappaValue=cohen_kappa_score(i.anomaly, y_pred)
  print("Kappa Value :",KappaValue)
  AUC=roc_auc_score(i.anomaly, y_pred)

  print("AUC         :",AUC)

  print("F1-Score Val  : ",f1_score(i.anomaly, y_pred))

  print("_______________________________________")

# Simulation

In [ ]:
def simulation(
    loss,
    model,
    patience,
    epochs,
    n:int = 0,
    end:int = 30,
    optimizer="adam",
    test_size = 0.2,
    batch_size=16,
    validation_batch_size = 16,
    delta = 0.5
    ):
  
  stopped_epochs = []
  auc_values_roc = []
  auc_values_recall_prec = []
  f1_values = []

  if loss == "focal_loss":
    loss = sm.losses.BinaryFocalLoss()
    print("Start the Simulation:")
    while(True):
      print("Lossfunction: ", loss),
      print("Iteration: ",n)
      #Step 1: Splitt data
      train_data, test_data, train_labels, test_labels = train_test_split(
        data,
        labels_bool,
        test_size=test_size
        )
      train_data_no_validation, train_data_validation, train_data_no_validation_labels, train_data_validation_labels = train_test_split(
        train_data,
        train_labels,
        test_size = test_size
        )
      
      #Step2: Train data
      model = model
      early_stopping = tf.keras.callbacks.EarlyStopping(patience=patience,
                                                      restore_best_weights=True) #generate callback
      model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
      history = model.fit(train_data,
                      train_data,
                      batch_size=batch_size,
                      epochs = epochs,
                      validation_data=(train_data_validation, train_data_validation),
                      callbacks=[
                          early_stopping,
                          ],
                      verbose = 0,
                      validation_batch_size = validation_batch_size,
                      shuffle=True)
      stopped_epochs.append(get_last_epoch(history)) #append stopped epoch 

      #Step3: Evaluate test data with AUC
      error_df = gen_error_df(model)
      fpr, tpr, thresholds = roc_curve(error_df.anomaly, error_df.reconstruction_error)
      auc_values_roc.append(auc(fpr, tpr))

      #Step4: Evaluate test data with AUC
      prec, recall, thresholds = precision_recall_curve(error_df["anomaly"], error_df["reconstruction_error"])
      auc_values_recall_prec.append(auc(recall, prec))
      best_idx = np.argmin(np.abs(prec-recall)[:-4])
      best_prec, best_recall = prec[best_idx], recall[best_idx]
      threshold = thresholds[best_idx]
      y_pred =  [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
      f1_values.append(f1_score(error_df.anomaly, y_pred))

      if (n >= end):
        print("Final Output:")
        print("Return stopped epochs per iteration, ")
        print("Return auc values per iteration, ")
        print("Return precision scores with best thresholds per iteration, ")
        print("Return recall scores with best thresholds per iteration, ")
        print("Return f1 scores with best thresholds per iteration, ")
        return(stopped_epochs,auc_values_roc,auc_values_recall_prec,f1_values)
        break
      n+=1
  
  elif loss == "huber_loss":
    loss = tf.keras.losses.Huber(delta=delta, reduction="auto", name="huber_loss")
    print("Start the Simulation:")
    while(True):
      print("Lossfunction: ", loss),
      print("Iteration: ",n)
      #Step 1: Splitt data
      train_data, test_data, train_labels, test_labels = train_test_split(
        data,
        labels_bool,
        test_size=test_size
        )
      train_data_no_validation, train_data_validation, train_data_no_validation_labels, train_data_validation_labels = train_test_split(
        train_data,
        train_labels,
        test_size = test_size
        )
      
      #Step2: Train data
      model = model
      early_stopping = tf.keras.callbacks.EarlyStopping(patience=patience,
                                                      restore_best_weights=True) #generate callback
      model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
      history = model.fit(train_data,
                      train_data,
                      batch_size=batch_size,
                      epochs = epochs,
                      validation_data=(train_data_validation, train_data_validation),
                      callbacks=[
                          early_stopping,
                          ],
                      verbose = 0,
                      validation_batch_size = validation_batch_size,
                      shuffle=True)
      stopped_epochs.append(get_last_epoch(history)) #append stopped epoch 

      #Step3: Evaluate test data with AUC
      error_df = gen_error_df(model)
      fpr, tpr, thresholds = roc_curve(error_df.anomaly, error_df.reconstruction_error)
      auc_values_roc.append(auc(fpr, tpr))

      #Step4: Evaluate test data with AUC
      prec, recall, thresholds = precision_recall_curve(error_df["anomaly"], error_df["reconstruction_error"])
      auc_values_recall_prec.append(auc(recall, prec))
      best_idx = np.argmin(np.abs(prec-recall)[:-4])
      best_prec, best_recall = prec[best_idx], recall[best_idx]
      threshold = thresholds[best_idx]
      y_pred =  [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
      f1_values.append(f1_score(error_df.anomaly, y_pred))

      if (n >= end):
        print("Final Output:")
        print("Return stopped epochs per iteration, ")
        print("Return auc values per iteration, ")
        print("Return precision scores with best thresholds per iteration, ")
        print("Return recall scores with best thresholds per iteration, ")
        print("Return f1 scores with best thresholds per iteration, ")
        return(stopped_epochs,auc_values_roc,auc_values_recall_prec,f1_values)
        break
      n+=1
      
  else:
    print("Start the Simulation:")
    while(True):
      print("Lossfunction: ", loss),
      print("Iteration: ",n)
      #Step 1: Splitt data
      train_data, test_data, train_labels, test_labels = train_test_split(
        data,
        labels_bool,
        test_size=test_size
        )
      train_data_no_validation, train_data_validation, train_data_no_validation_labels, train_data_validation_labels = train_test_split(
        train_data,
        train_labels,
        test_size = test_size
        )
      #Step2: Train data
      model = model
      early_stopping = tf.keras.callbacks.EarlyStopping(patience=patience,
                                                      restore_best_weights=True) #generate callback
      model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
      history = model.fit(train_data,
                      train_data,
                      batch_size=batch_size,
                      epochs = epochs,
                      validation_data=(train_data_validation, train_data_validation),
                      callbacks=[
                          early_stopping,
                          ],
                      verbose = 0,
                      validation_batch_size = validation_batch_size,
                      shuffle=True)
      stopped_epochs.append(get_last_epoch(history)) #append stopped epoch
      
      #Step3: Evaluate test data with AUC
      error_df = gen_error_df(model)
      fpr, tpr, thresholds = roc_curve(error_df.anomaly, error_df.reconstruction_error)
      auc_values_roc.append(auc(fpr, tpr))

      #Step4: Evaluate test data with AUC
      prec, recall, thresholds = precision_recall_curve(error_df["anomaly"], error_df["reconstruction_error"])
      auc_values_recall_prec.append(auc(recall, prec))
      best_idx = np.argmin(np.abs(prec-recall)[:-4])
      best_prec, best_recall = prec[best_idx], recall[best_idx]
      threshold = thresholds[best_idx]
      y_pred =  [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
      f1_values.append(f1_score(error_df.anomaly, y_pred))

      if (n >= end):
        print("Final Output:")
        print("Return stopped epochs per iteration, ")
        print("Return auc values per iteration, ")
        print("Return precision scores with best thresholds per iteration, ")
        print("Return recall scores with best thresholds per iteration, ")
        print("Return f1 scores with best thresholds per iteration, ")
        return(stopped_epochs,auc_values_roc,auc_values_recall_prec,f1_values)
        break
      n+=1

In [ ]:
simulation_mse = simulation(model = model_mse,
                            end = 30,
                            patience = 10, #EaryStopping after 10 epochs,
                            epochs = 1000,
                            loss = "mean_squared_error")
with open("simulation_mse_thyroid_.txt", "w") as f:
   # Convert the list to a string and write it to the file
    f.write(str(simulation_mse))

In [ ]:
simulation_mae = simulation(model = model_mae,
                            end = 30,
                            patience = 10, #EaryStopping after 10 epochs,
                            epochs = 1000,
                            loss = "mean_absolute_error")
with open("simulation_mae_thyroid_.txt", "w") as f:
   # Convert the list to a string and write it to the file
    f.write(str(simulation_mae))

In [ ]:
simulation_huber = simulation(model = model_huber,
                            end = 30,
                            patience = 10, #EaryStopping after 10 epochs,
                            epochs = 1000,
                            loss = "huber_loss",
                            delta = 0.5)
with open("simulation_huber_thyroid_.txt", "w") as f:
   # Convert the list to a string and write it to the file
    f.write(str(simulation_huber))

In [ ]:
simulation_ce= simulation(model = model_ce,
                          end = 30,
                          patience = 10, #EaryStopping after 10 epochs,
                          epochs = 1000,
                          loss = "binary_crossentropy")
with open("simulation_ce_thyroid_.txt", "w") as f:
   # Convert the list to a string and write it to the file
    f.write(str(simulation_ce))

In [ ]:
simulation_focal =simulation(model = model_focal,
                             end = 30,
                             patience = 10, #EaryStopping after 10 epochs,
                             epochs = 1000,
                             loss ="focal_loss")
with open("simulation_focal_thyroid_.txt", "w") as f:
   # Convert the list to a string and write it to the file
    f.write(str(simulation_focal))

In [ ]:
for i in [simulation_mse, simulation_mae, simulation_huber, simulation_ce, simulation_focal]:
  print("_______________________________________________________")
  print("Mean and standard deviation of last epochs")
  print(np.asarray(i[0]).mean())
  print(np.asarray(i[0]).std())
  print("Mean and standard deviation of ROC-AUC")
  print(np.asarray(i[1]).mean())
  print(np.asarray(i[1]).std())
  print("Mean and standard deviation of Recall-Precision-AUC")
  print(np.asarray(i[2]).mean())
  print(np.asarray(i[2]).std())



# Statistische Evaluation

Die Ergebnisse aus der Simulation wurden in Textfiles gespeichert. Um die Daten zu laden, wurden diese manuell hier herein kopiert. 

Quelle:


*   simulation_mse_thyroid_.txt

*   simulation_mae_thyroid_.txt

*   simulation_huber_thyroid_.txt

*   simulation_ce_thyroid_.txt

*   simulation_focal_thyroid_.txt

Um die Resultate der Ausarbeitung zu sehen, den folgenden Block auskommentieren. Ansonsten kann eine neue Simulation verwendet werden (GPU-Unterstützung sollte dafür aktiv sein).


In [ ]:
# simulation_mse = [[20, 57, 59, 22, 34, 76, 23, 21, 40, 25, 21, 33, 291, 15, 23, 12, 23, 17, 17, 24, 29, 12, 20, 14, 23, 10, 18, 14, 13, 24, 17], [0.9584652495100257, 0.9482134780642245, 0.9550731192522237, 0.9504748982360922, 0.9504748982360922, 0.9491180461329716, 0.9428614503241368, 0.9481380973918287, 0.9408261721694559, 0.9419568822553898, 0.9388662746871702, 0.9421830242725765, 0.9386401326699835, 0.935549525101764, 0.9299713553444897, 0.9338157696366651, 0.9249208502939845, 0.9293683099653249, 0.9353233830845772, 0.9308005427408412, 0.9289160259309514, 0.9412030755314338, 0.9363787124981154, 0.9326096788783357, 0.9240916628976331, 0.9402985074626866, 0.9314789687924017, 0.9338911503090609, 0.9329865822403136, 0.9292929292929293, 0.9338911503090608], [0.6803622816065465, 0.663316492547374, 0.6888730415670026, 0.6814040351241367, 0.6834342236647633, 0.6826546619267884, 0.6761305895668994, 0.678256333960623, 0.6630007678591426, 0.6629701336462936, 0.6703576011098198, 0.67949454868509, 0.6458677966603843, 0.6410032608543998, 0.6357026214930742, 0.6334190853697573, 0.6293122570155172, 0.6517539583341299, 0.6507019447303317, 0.6291399458411626, 0.6335503762586893, 0.6452104248009196, 0.6395622819907396, 0.637447461518958, 0.6213624373832813, 0.6396836491549408, 0.634711612833963, 0.6410524633749393, 0.6331438351016214, 0.6215217463335303, 0.6258085898635214], [0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.5714285714285715, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.5714285714285715, 0.5714285714285715]]
# simulation_mae = [[89, 21, 36, 10, 20, 45, 31, 27, 92, 68, 16, 20, 11, 18, 19, 31, 22, 18, 19, 32, 13, 48, 12, 10, 23, 18, 23, 19, 21, 40, 34], [0.9599728629579376, 0.9604251469923112, 0.9592190562339816, 0.9604251469923112, 0.9605005276647068, 0.9624604251469924, 0.9601990049751243, 0.959897482285542, 0.9497210915121362, 0.9503995175636968, 0.9469320066334991, 0.9454997738579829, 0.9457259158751696, 0.9448967284788181, 0.9458766772199608, 0.9426353083069501, 0.9443690637720489, 0.9459520578923564, 0.9444444444444445, 0.9431629730137193, 0.9454243931855871, 0.9422584049449721, 0.9397708427559174, 0.94451982511684, 0.9411276948590382, 0.9412030755314337, 0.942107643600181, 0.9442183024272577, 0.9364540931705111, 0.9412784562038293, 0.9401477461178953], [0.6678745215784142, 0.6719428380727064, 0.6681782331031876, 0.6703828137591066, 0.6722304268834298, 0.6879151427645401, 0.6850718540525206, 0.6849163848307965, 0.6736557024591323, 0.6843127534757543, 0.679085122721295, 0.6689502025007594, 0.6799215329765188, 0.6783629452337405, 0.6799845470065657, 0.677004407429214, 0.6803941245288192, 0.6815787650831574, 0.6800045064248209, 0.6741273365803461, 0.6817956085614879, 0.675843948511906, 0.6636297071028537, 0.6717767219145767, 0.6754487778463963, 0.6669341759647384, 0.6639329172443394, 0.6780901811049225, 0.6575941582524741, 0.6751936815370715, 0.6625297726519677], [0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287, 0.6285714285714287]]
# simulation_huber = [[85, 15, 20, 12, 14, 16, 13, 17, 13, 18, 12, 21, 10, 12, 11, 25, 35, 10, 14, 15, 18, 22, 12, 26, 19, 28, 45, 13, 16, 18, 27], [0.9404504504504504, 0.9411711711711712, 0.941081081081081, 0.9427927927927928, 0.941981981981982, 0.940900900900901, 0.9409909909909909, 0.9423423423423423, 0.9434234234234233, 0.9405405405405406, 0.9418918918918919, 0.9420720720720721, 0.9436036036036036, 0.942072072072072, 0.9406306306306306, 0.9436936936936937, 0.9423423423423424, 0.9408108108108109, 0.9416216216216217, 0.9436936936936936, 0.9422522522522522, 0.9435135135135135, 0.9416216216216217, 0.9428828828828829, 0.941081081081081, 0.9412612612612612, 0.9425225225225224, 0.9403603603603603, 0.9428828828828828, 0.9424324324324324, 0.9424324324324324], [0.40314085664402444, 0.40405685453775664, 0.40306419780742486, 0.41047298536483423, 0.4080883612772183, 0.40261550786900147, 0.402627242408175, 0.4078329362331152, 0.4113737290515094, 0.4029954697071223, 0.40765395463076354, 0.4080996974079129, 0.4128830067085303, 0.40748201752519353, 0.4025218163629189, 0.41363514050712513, 0.40763856263088266, 0.4034278036904077, 0.40567846903441107, 0.41533184306950427, 0.407821443210319, 0.4128714340154459, 0.4040425865477913, 0.41048439953723775, 0.40306419780742486, 0.40342141617034305, 0.4083670960363105, 0.40331209373607296, 0.4117069454052546, 0.40958287973863566, 0.406407302649005], [0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965, 0.3448275862068965]]
# simulation_ce = [[145, 10, 15, 14, 35, 25, 34, 10, 20, 24, 44, 30, 63, 10, 16, 41, 26, 19, 33, 44, 32, 13, 23, 25, 16, 22, 55, 11, 18, 69, 17], [0.9473873873873874, 0.946036036036036, 0.9427027027027026, 0.9467567567567567, 0.9435135135135135, 0.9399099099099099, 0.9374774774774774, 0.936036036036036, 0.940990990990991, 0.9227027027027027, 0.9345045045045046, 0.9351351351351352, 0.9300900900900901, 0.9309909909909909, 0.9312612612612612, 0.9214414414414414, 0.9266666666666667, 0.9272972972972973, 0.9247747747747747, 0.9033333333333334, 0.9193693693693694, 0.9186486486486487, 0.9143243243243243, 0.9146846846846848, 0.9068468468468468, 0.9136936936936937, 0.9281981981981982, 0.9211711711711712, 0.9255855855855857, 0.9181981981981983, 0.9082882882882883], [0.305232232134608, 0.3060093262934467, 0.3067543282698745, 0.32179274126900254, 0.2849302560675358, 0.29850240794925537, 0.2894465541026962, 0.29882737536690873, 0.3024876655313838, 0.28851004053851725, 0.3411958784821764, 0.3314869967298828, 0.2972532460692812, 0.31121481031815756, 0.2936031093953547, 0.3131945755557457, 0.27996722353507775, 0.2962731231384651, 0.3124522041552719, 0.28602341774935497, 0.3039241425375489, 0.2965962165396764, 0.3023627396101569, 0.3052906105842364, 0.2843951143015883, 0.28491183959468747, 0.26348066438113915, 0.3067816610138892, 0.2999366601739338, 0.29335017000454966, 0.293613518788803], [0.4137931034482759, 0.4137931034482759, 0.4137931034482759, 0.4137931034482759, 0.3448275862068965, 0.4137931034482759, 0.3448275862068965, 0.4137931034482759, 0.4137931034482759, 0.3448275862068965, 0.3448275862068965, 0.2758620689655172, 0.2758620689655172, 0.2758620689655172, 0.2758620689655172, 0.2758620689655172, 0.2758620689655172, 0.2758620689655172, 0.2758620689655172, 0.2758620689655172, 0.3448275862068965, 0.2758620689655172, 0.2758620689655172, 0.3448275862068965, 0.3448275862068965, 0.2758620689655172, 0.4137931034482759, 0.4137931034482759, 0.4137931034482759, 0.2758620689655172, 0.2758620689655172]]
# simulation_focal = [[74, 10, 10, 21, 154, 47, 24, 37, 25, 16, 17, 11, 12, 22, 24, 21, 18, 29, 27, 19, 17, 18, 24, 12, 30, 28, 23, 19, 11, 22, 13], [0.9942329157881681, 0.9939848691554012, 0.9942329157881682, 0.9944809624209352, 0.9489644053081979, 0.9451196825003101, 0.9371821902517673, 0.9474141138534046, 0.9477241721443632, 0.9492744635991567, 0.9505146967629914, 0.9507007317375666, 0.9466079622969117, 0.9478481954607467, 0.9476621604861714, 0.9502046384720327, 0.9466699739551037, 0.9452437058166936, 0.9481582537517054, 0.9406548431105047, 0.9486543470172392, 0.9492124519409649, 0.9427012278308323, 0.9507627433957584, 0.9445615775765844, 0.945863822398611, 0.945615775765844, 0.9433213444127495, 0.9389185166811361, 0.9474761255115962, 0.9461118690313779], [0.8698570934196981, 0.8661674489131845, 0.8698570934196981, 0.878098589235075, 0.6759791622277223, 0.6790081210966336, 0.6554287150141308, 0.626739849636415, 0.6313268783056842, 0.6442071922515911, 0.605200587829997, 0.6053092100425035, 0.6383547864345533, 0.6204459493622417, 0.6249453083302469, 0.6175003688732906, 0.6195452729096073, 0.6117432165085721, 0.6137142402183011, 0.5965431836180759, 0.6328180280780563, 0.6337953085411974, 0.610753014773887, 0.675614127133818, 0.6030999672537131, 0.6263101907101767, 0.6211278125010302, 0.614412748610864, 0.612225604697942, 0.6387403071531994, 0.61367731614722], [0.8372093023255814, 0.8372093023255814, 0.8372093023255814, 0.8837209302325582, 0.6511627906976744, 0.6976744186046512, 0.6511627906976744, 0.6046511627906977, 0.6046511627906977, 0.6511627906976744, 0.6046511627906977, 0.6046511627906977, 0.6046511627906977, 0.6511627906976744, 0.6046511627906977, 0.6046511627906977, 0.6046511627906977, 0.6046511627906977, 0.6046511627906977, 0.6046511627906977, 0.6511627906976744, 0.6976744186046512, 0.6046511627906977, 0.6511627906976744, 0.6046511627906977, 0.6046511627906977, 0.6046511627906977, 0.6046511627906977, 0.6046511627906977, 0.6046511627906977, 0.6046511627906977]]


Der letzte Index inhät Daten zum erzielten F1-Score. Dieser wird allerdings nicht weiter ausgewertet. Stattdessen wird der AUC für die Precision-Recall Curve weiter betrachtet.

In [ ]:
summary = [
    simulation_mse[0],
    simulation_mae[0],
    simulation_huber[0],
    simulation_ce[0],
    simulation_focal[0],
    simulation_mse[1],
    simulation_mae[1],
    simulation_huber[1],
    simulation_ce[1],
    simulation_focal[1],
    simulation_mse[2],
    simulation_mae[2],
    simulation_huber[2],
    simulation_ce[2],
    simulation_focal[2]   
]

In [ ]:
results = pd.DataFrame(zip(*summary),
                       columns=[
                           "last_epochs_mse",
                           "last_epochs_mae",
                           "last_epochs_huber",
                           "last_epochs_ce",
                           "last_epochs_focal",
                           "auc_mse",
                           "auc_mae",
                           "auc_huber",
                           "auc_ce",
                           "auc_focal",
                           "auprc_mse",
                           "auprc_mae",
                           "auprc_huber",
                           "auprc_ce",
                           "auprc_focal"
                       ]

)

results

In [ ]:
#print boxplot for each variable
fig, axs = plt.subplots(ncols=5, nrows=3, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in results.items():
    sns.boxplot(y=k, data=results, ax=axs[index])
    index += 1
save_fig("boxplots_results")
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

## Kruskal-Wallis-Test

Der [Kruskal-Wallis-Test](https:https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kruskal.html//) ist ein nichtparametrischer Test, der zum Vergleich der Mittelwerte von drei oder mehr unabhängigen Gruppen verwendet werden kann. Er ähnelt der einseitigen ANOVA, setzt aber nicht voraus, dass die Daten normal verteilt sind.

Zur Durchführung des Kruskal-Wallis-Tests werden die Daten zunächst vom niedrigsten zum höchsten Wert geordnet. Die Ränge werden dann zur Berechnung einer Teststatistik verwendet, die mit einem kritischen Wert verglichen wird, um festzustellen, ob der Unterschied zwischen den Gruppenmitteln statistisch signifikant ist.

Der Kruskal-Wallis-Test ist geeignet, wenn die zu vergleichenden Populationen unabhängig sind und die Daten ordinal (d. h., sie können in eine Rangfolge gebracht werden) oder kontinuierlich (aber nicht normalverteilt) sind. Er ist auch geeignet, wenn die Stichprobengröße klein oder ungleich ist.



Last-Epochs: Gibt es Unterschiede bezüglich Konvergenz? 

In [ ]:
for i in [simulation_mse, simulation_mae, simulation_huber, simulation_ce, simulation_focal]:
  print("_______________________________________________________")
  print("Mean and standard deviation of last epochs")
  print(np.asarray(i[0]).mean())
  print(np.asarray(i[0]).std())

In [ ]:
import scipy.stats as stats
'''
Null hypothesis that the population median of all of the groups are equal

Here: stopped epochs to evaluate the convergence ability 
'''
stats.kruskal(
    np.hstack(simulation_mse[0]),
    np.hstack(simulation_mae[0]),
    np.hstack(simulation_huber[0]),
    np.hstack(simulation_ce[0]),
    np.hstack(simulation_focal[0]),
     )
# KruskalResult(statistic=22.23847083667068, pvalue=0.0001796605135019599)


AUC-ROC Gibt es Unterschiede für die Performance in den Testdaten?

In [ ]:
for i in [simulation_mse, simulation_mae, simulation_huber, simulation_ce, simulation_focal]:
  print("_______________________________________________________")
  print("Mean and standard deviation of roc_auc")
  print(np.asarray(i[1]).mean())
  print(np.asarray(i[1]).std())

In [ ]:
import scipy.stats as stats
'''
Null hypothesis that the population median of all of the groups are equal

Here: stopped epochs to evaluate the convergence ability 
'''
stats.kruskal(
    np.hstack(simulation_mse[1]),
    np.hstack(simulation_mae[1]),
    np.hstack(simulation_huber[1]),
    np.hstack(simulation_ce[1]),
    np.hstack(simulation_focal[1]),
     )
 # KruskalResult(statistic=131.99214948486943, pvalue=1.459883736690209e-27)


AUC-Precision-Recall-Curve: Gibt es Unterschiede für die Performance in den Testdaten?

In [ ]:
for i in [simulation_mse, simulation_mae, simulation_huber, simulation_ce, simulation_focal]:
  print("_______________________________________________________")
  print("Mean and standard deviation of roc_recall_precision")
  print(np.asarray(i[2]).mean())
  print(np.asarray(i[2]).std())

In [ ]:
import scipy.stats as stats
'''
Null hypothesis that the population median of all of the groups are equal

Here: stopped epochs to evaluate the convergence ability 
'''
stats.kruskal(
    np.hstack(simulation_mse[2]),
    np.hstack(simulation_mae[2]),
    np.hstack(simulation_huber[2]),
    np.hstack(simulation_ce[2]),
    np.hstack(simulation_focal[2]),
     )
# KruskalResult(statistic=131.86117955510082, pvalue=1.557160300415864e-27)


Wenn die Nullhypothese des Kruskal-Wallis-Tests abgelehnt wird, bedeutet dies, dass sich mindestens einer der Gruppenmittelwerte signifikant von den anderen unterscheidet. Der Kruskal-Wallis-Test sagt jedoch nicht aus, welche Gruppe(n) sich unterscheiden. Um festzustellen, welche Gruppe(n) sich unterscheidet/unterscheiden, muss eine zusätzliche Analyse durchgeführt werden, z. B. ein Post-hoc-Test.

Es gibt mehrere Post-hoc-Tests, die zum Vergleich der Mittelwerte bestimmter Gruppenpaare verwendet werden können. Zu den gängigen Tests gehören der [Dunn-Test](https://scikit-posthocs.readthedocs.io/en/latest/generated/scikit_posthocs.posthoc_dunn), der [Conover-Test](https://scikit-posthocs.readthedocs.io/en/latest/generated/scikit_posthocs.posthoc_conover) und der [Steel-Dwass-Test](https://scikit-posthocs.readthedocs.io/en/latest/generated/scikit_posthocs.posthoc_dscf). Diese Tests verwenden die Rangdaten aus dem Kruskal-Wallis-Test, um festzustellen, welche Gruppenpaare signifikant unterschiedliche Mittelwerte aufweisen.

Alle diese Post-hoc-Tests können verwendet werden, um festzustellen, welche Gruppenpaare nach Durchführung des Kruskal-Wallis-Tests signifikant unterschiedliche Mittelwerte aufweisen.



In [ ]:
pip install scikit-posthocs

### [Dunn-Test](https://scikit-posthocs.readthedocs.io/en/latest/generated/scikit_posthocs.posthoc_dunn)

Der Dunn-Test ist ein Post-hoc-Test, der die Mittelwerte aller Gruppenpaare unter Verwendung der Rangdaten aus dem Kruskal-Wallis-Test vergleicht. Er basiert auf der Differenz der Ränge zwischen den beiden zu vergleichenden Gruppen und passt sich durch Kontrolle der Falschentdeckungsrate an Mehrfachvergleiche an.

In [ ]:
# last-epochs
import scikit_posthocs as sp

last_epochs = [np.hstack(simulation_mse[0]),
    np.hstack(simulation_mae[0]),
    np.hstack(simulation_huber[0]),
    np.hstack(simulation_ce[0]),
    np.hstack(simulation_focal[0])]

sp.posthoc_dunn(last_epochs, p_adjust = 'holm')

Beobachtung: Keine signifikanten Unterschiede bzgl. der Konvergenz zu beobachten.



In [ ]:
# roc_auc
import scikit_posthocs as sp

roc_auc = [np.hstack(simulation_mse[1]),
    np.hstack(simulation_mae[1]),
    np.hstack(simulation_huber[1]),
    np.hstack(simulation_ce[1]),
    np.hstack(simulation_focal[1])]

sp.posthoc_dunn(roc_auc, p_adjust = 'holm')

In [ ]:
# recall_precision_auc
import scikit_posthocs as sp

recall_precision_auc = [np.hstack(simulation_mse[2]),
    np.hstack(simulation_mae[2]),
    np.hstack(simulation_huber[2]),
    np.hstack(simulation_ce[2]),
    np.hstack(simulation_focal[2])]

sp.posthoc_dunn(recall_precision_auc, p_adjust = 'holm')

# Endresultat für Thyroid Disease

| Performance                  | Loss       |
|------------------------------|------------|
| Schnellste Konvergenz?       | -          |
| Bester ROC-AUC?              | MAE & FL   |
| Bester ROC-Recall-Precision? | MAE & MSE  |

Die Nullhypothese des Kruksal-Wallis-Test bzgl. Konvergenz (gestoppte Epochen) konnte nicht verworfen werden.

Der MAE & FL haben signifikant die besten Ergebnisse gezeigt mit der AUC und unterscheiden sich nicht untereinander. Diese Unterscheiden sich nur signifikant zu den drei anderen Modellen. Somit wird die Entscheidung getroffen, dass beide Modelle im Schnitt am besten performt haben mit der AUC-Metrik.

Der MAE & MSE haben signifikant die besten Ergebnisse gezeigt mit der AUPRC-Metrik und unterscheiden sich nicht untereinander. Diese Unterscheiden sich nur signifikant zu den drei anderen Modellen. Somit wird die Entscheidung getroffen, dass beide Modelle im Schnitt am besten performt haben mit der AUPRC-Metrik.

Da der AUC und der AUPRC mit dem MAE im Schnitt am besten gewesen sind, wird nach dieser Simulation das MAE-Modell als finales Modell gewählt.